In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold
from lightgbm import LGBMClassifier,LGBMRegressor
import lightgbm as lgb
import tensorflow as tf
import dask.dataframe as dd
import math
import random
import os
import gc
import sys
plt.style.use('fivethirtyeight')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [2]:
['item_id', 'dept_id', 'store_id', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'sales_lag_28', 'sales_lag_28_roll_mean_7', 'sales_lag_28_roll_std_7', 'sales_lag_28_roll_max_7', 'sales_lag_28_roll_mean_28', 'sales_lag_28_roll_max_28', 'sales_lag_91', 'sales_lag_91_roll_mean_7', 'sales_lag_91_roll_std_7', 'sales_lag_91_roll_mean_28', 'turnover_lag_28', 'turnover_lag_28_roll_mean_7', 'turnover_lag_91', 'turnover_lag_91_roll_mean_7', 'turnover_lag_91_roll_std_7', 'turnover_lag_91_roll_max_7', 'is_sport', 'is_holiday', 'sales_lag_28_roll_mean_91', 'sales_lag_28_roll_std_91', 'sales_lag_28_roll_max_91', 'sales_lag_28_roll_mean_182', 'sales_lag_28_roll_std_182', 'sales_lag_91_roll_mean_91', 'sales_lag_91_roll_max_91', 'sales_lag_91_roll_mean_182', 'sales_lag_182', 'sales_lag_182_roll_mean_182', 'sales_lag_182_roll_std_182', 'sales_lag_182_roll_max_182', 'sales_lag_364', 'sales_lag_364_roll_mean_7', 'sales_lag_364_roll_std_7', 'sales_lag_364_roll_max_7', 'sales_lag_364_roll_mean_28', 'sales_lag_364_roll_std_28', 'sales_lag_364_roll_max_28']

['item_id',
 'dept_id',
 'store_id',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'sell_price',
 'sales_lag_28',
 'sales_lag_28_roll_mean_7',
 'sales_lag_28_roll_std_7',
 'sales_lag_28_roll_max_7',
 'sales_lag_28_roll_mean_28',
 'sales_lag_28_roll_max_28',
 'sales_lag_91',
 'sales_lag_91_roll_mean_7',
 'sales_lag_91_roll_std_7',
 'sales_lag_91_roll_mean_28',
 'turnover_lag_28',
 'turnover_lag_28_roll_mean_7',
 'turnover_lag_91',
 'turnover_lag_91_roll_mean_7',
 'turnover_lag_91_roll_std_7',
 'turnover_lag_91_roll_max_7',
 'is_sport',
 'is_holiday',
 'sales_lag_28_roll_mean_91',
 'sales_lag_28_roll_std_91',
 'sales_lag_28_roll_max_91',
 'sales_lag_28_roll_mean_182',
 'sales_lag_28_roll_std_182',
 'sales_lag_91_roll_mean_91',
 'sales_lag_91_roll_max_91',
 'sales_lag_91_roll_mean_182',
 'sales_lag_182',
 'sales_lag_182_roll_mean_182',
 'sales_lag_182_roll_std_182',
 'sales_lag_182_roll_max_182',
 'sales_lag_364',
 'sales_lag_364_roll_mean_7',
 'sales_lag_364_roll_std_7',
 'sales_lag_364_roll_max_

In [3]:
['turnover_lag_364','turnover_lag_182','turnover_lag_364_roll_std_91',
 'turnover_lag_182_roll_mean_182','turnover_lag_91_roll_mean_182',
'turnover_lag_182_roll_std_182','turnover_lag_28_roll_mean_91']

['turnover_lag_364',
 'turnover_lag_182',
 'turnover_lag_364_roll_std_91',
 'turnover_lag_182_roll_mean_182',
 'turnover_lag_91_roll_mean_182',
 'turnover_lag_182_roll_std_182',
 'turnover_lag_28_roll_mean_91']

In [4]:
used_index = [9747090,12689405,17844519,40666139,52063828]

In [5]:
from tqdm import tqdm
tqdm.pandas()

In [6]:
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [7]:
seed_everything(seed=51)

In [8]:
%%time
data = pd.read_pickle('data.pkl')

CPU times: user 21 s, sys: 6.56 s, total: 27.5 s
Wall time: 27.5 s


In [9]:
data['sold'] = data['sell_price'].notnull() * 1

In [10]:
data.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,sold
60034805,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,NaN,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.980469,1
60034806,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,NaN,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.480469,1
60034807,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,NaN,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,3.980469,1
60034808,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,NaN,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.280273,1
60034809,FOODS_3_827_WI_3,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1969,NaN,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.000000,1


In [11]:
#Sales Lag Creation
for i in tqdm([28,56,91,182,364,728]):
    data['sales_lag_{}'.format(i)] = data.groupby('id')['sales'].shift(i)

100%|██████████| 6/6 [01:16<00:00, 12.73s/it]


In [12]:
%%time
data['turnover'] = data['sales'] * data['sell_price']
data['turnover'] = data['turnover'].fillna(0.0)

CPU times: user 1.92 s, sys: 132 ms, total: 2.05 s
Wall time: 2.05 s


In [13]:
#Turnover Lag Creation
for i in tqdm([28,56,91,182,364,728]):
    data['turnover_lag_{}'.format(i)] = data.groupby('id')['turnover'].shift(i)

100%|██████████| 6/6 [01:04<00:00, 10.75s/it]


In [14]:
#Sales Aggregation
i,j = 28,7
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
data['sales_lag_{}_roll_max_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).max())
print("*"*50)
i,j = 28,28
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_max_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).max())
print("*"*50)
i,j = 28,91
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
data['sales_lag_{}_roll_max_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).max())
print("*"*50)
i,j = 28,182
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
print("*"*50)
i,j = 91,7
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
print("*"*50)
i,j = 91,28
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
print("*"*50)
i,j = 91,91
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
print("*"*50)
i,j = 91,182
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
print("*"*50)
i,j = 182,182
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
print("*"*50)
i,j = 364,7
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
data['sales_lag_{}_roll_max_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).max())
print("*"*50)
i,j = 364,28
data['sales_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['sales_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
data['sales_lag_{}_roll_max_{}'.format(i,j)]= data.groupby('id')['sales_lag_{}'.format(i)].transform(lambda x : x.rolling(j).max())
print("*"*50)

**************************************************
**************************************************
**************************************************
**************************************************
**************************************************
**************************************************
**************************************************
**************************************************
**************************************************
**************************************************
**************************************************


In [15]:
#Turnover Aggregation
i,j = 28,7
data['turnover_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
print("*"*50)
i,j = 28,91
data['turnover_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
print("*"*50)
i,j = 91,7
data['turnover_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['turnover_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
data['turnover_lag_{}_roll_max_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).max())
print("*"*50)
i,j = 91,182
data['turnover_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
print("*"*50)
i,j = 182,182
data['turnover_lag_{}_roll_mean_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).mean())
data['turnover_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
print("*"*50)
i,j = 364,91
data['turnover_lag_{}_roll_std_{}'.format(i,j)]= data.groupby('id')['turnover_lag_{}'.format(i)].transform(lambda x : x.rolling(j).std())
print("*"*50)

**************************************************
**************************************************
**************************************************
**************************************************
**************************************************
**************************************************


In [16]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [17]:
data = reduce_mem_usage(data)

Mem. usage decreased to 11679.75 Mb (3.3% reduction)


In [18]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,sold,sales_lag_28,sales_lag_56,sales_lag_91,sales_lag_182,sales_lag_364,sales_lag_728,turnover,turnover_lag_28,turnover_lag_56,turnover_lag_91,turnover_lag_182,turnover_lag_364,turnover_lag_728,sales_lag_28_roll_mean_7,sales_lag_28_roll_std_7,sales_lag_28_roll_max_7,sales_lag_28_roll_mean_28,sales_lag_28_roll_max_28,sales_lag_28_roll_mean_91,sales_lag_28_roll_std_91,sales_lag_28_roll_max_91,sales_lag_28_roll_mean_182,sales_lag_28_roll_std_182,sales_lag_91_roll_mean_7,sales_lag_91_roll_std_7,sales_lag_91_roll_mean_28,sales_lag_91_roll_mean_91,sales_lag_91_roll_mean_182,sales_lag_182_roll_mean_182,sales_lag_182_roll_std_182,sales_lag_364_roll_mean_7,sales_lag_364_roll_std_7,sales_lag_364_roll_max_7,sales_lag_364_roll_mean_28,sales_lag_364_roll_std_28,sales_lag_364_roll_max_28,turnover_lag_28_roll_mean_7,turnover_lag_28_roll_mean_91,turnover_lag_91_roll_mean_7,turnover_lag_91_roll_std_7,turnover_lag_91_roll_max_7,turnover_lag_91_roll_mean_182,turnover_lag_182_roll_mean_182,turnover_lag_182_roll_std_182,turnover_lag_364_roll_std_91
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
for i in tqdm(['item_id','dept_id','cat_id','store_id','state_id']):
    le = LabelEncoder()
    data[i] = le.fit_transform(data[i]) 

100%|██████████| 5/5 [01:12<00:00, 14.42s/it]


In [21]:
print(data.shape)
data = data[(data['sold'] == 1.0)]
print(data.shape)

(60034810, 64)
(47735397, 64)


In [22]:
data['day_number'] = data['d'].progress_apply(lambda x: int(x.split('_')[1]))

100%|██████████| 47735397/47735397 [01:03<00:00, 746262.09it/s]


In [23]:
print(data.shape)
data = data[(data['day_number'] >= 1069)]
print(data.shape)

(47735397, 65)
(26532479, 65)


In [24]:
other_list = ["Father's day","Mother's day","ValentinesDay","NewYear"]
cl1 = 'event_name_1'
cl2 = 'event_name_2'
data['is_other'] = (data[cl1].isin(other_list) * 1 + data[cl2].isin(other_list) * 1 > 0) *1
data['is_holiday'] = (data[cl1].notnull() * 1 + data[cl2].notnull() * 1 > 0) *1

In [25]:
data['dayofyear'] = pd.to_datetime(data['date']).dt.dayofyear
data['year'] = pd.to_datetime(data['date']).dt.year

In [26]:
data['month'] = data['date'].progress_apply(lambda x: x.split('-')[1])

100%|██████████| 26532479/26532479 [00:28<00:00, 927733.03it/s]


In [27]:
data['month'] = data['month'].progress_apply(int)

100%|██████████| 26532479/26532479 [00:26<00:00, 991428.02it/s] 


In [28]:
print(data.shape)
data = data[ data['month'] < 7 ]
print(data.shape)

(26532479, 70)
(15589620, 70)


In [29]:
del data['event_name_1'],data['event_type_1'],data['event_name_2'],data['event_type_2']
gc.collect()

78

In [30]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,sold,sales_lag_28,sales_lag_56,sales_lag_91,sales_lag_182,sales_lag_364,sales_lag_728,turnover,turnover_lag_28,turnover_lag_56,turnover_lag_91,turnover_lag_182,turnover_lag_364,turnover_lag_728,sales_lag_28_roll_mean_7,sales_lag_28_roll_std_7,sales_lag_28_roll_max_7,sales_lag_28_roll_mean_28,sales_lag_28_roll_max_28,sales_lag_28_roll_mean_91,sales_lag_28_roll_std_91,sales_lag_28_roll_max_91,sales_lag_28_roll_mean_182,sales_lag_28_roll_std_182,sales_lag_91_roll_mean_7,sales_lag_91_roll_std_7,sales_lag_91_roll_mean_28,sales_lag_91_roll_mean_91,sales_lag_91_roll_mean_182,sales_lag_182_roll_mean_182,sales_lag_182_roll_std_182,sales_lag_364_roll_mean_7,sales_lag_364_roll_std_7,sales_lag_364_roll_max_7,sales_lag_364_roll_mean_28,sales_lag_364_roll_std_28,sales_lag_364_roll_max_28,turnover_lag_28_roll_mean_7,turnover_lag_28_roll_mean_91,turnover_lag_91_roll_mean_7,turnover_lag_91_roll_std_7,turnover_lag_91_roll_max_7,turnover_lag_91_roll_mean_182,turnover_lag_182_roll_mean_182,turnover_lag_182_roll_std_182,turnover_lag_364_roll_std_91,day_number,is_other,is_holiday,dayofyear,year,month
32563320,HOBBIES_1_001_CA_1,1437,3,1,0,0,d_1069,1.0,2014-01-01,11349,1,1,0,8.257812,1,1.0,1.0,0.0,0.0,0.0,0.0,8.257812,8.257812,8.257812,0.000000,0.000000,0.000000,0.000000,1.000000,0.577148,2.0,0.714355,3.0,0.549316,0.703613,3.0,0.368164,0.623535,0.571289,0.786621,0.464355,0.329590,0.164795,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,8.257812,4.539062,4.718750,6.496094,16.515625,1.368164,0.000000,0.000000,0.000000,1069,1,1,1,2014,1
32563321,HOBBIES_1_002_CA_1,1438,3,1,0,0,d_1069,0.0,2014-01-01,11349,1,1,0,3.970703,1,0.0,0.0,0.0,2.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,7.941406,0.000000,3.970703,0.000000,0.000000,0.0,0.071411,1.0,0.120850,0.360107,2.0,0.203247,0.442627,0.428467,0.786621,0.142822,0.230713,0.236206,0.252686,0.495117,0.142822,0.377930,1.0,0.964355,1.169922,3.0,0.000000,0.479980,1.702148,3.125000,7.941406,0.937988,1.003906,1.965820,3.419922,1069,1,1,1,2014,1
32563323,HOBBIES_1_004_CA_1,1440,3,1,0,0,d_1069,2.0,2014-01-01,11349,1,1,0,4.640625,1,1.0,2.0,1.0,0.0,0.0,0.0,9.281250,4.640625,9.281250,4.640625,0.000000,0.000000,0.000000,3.142578,2.671875,8.0,2.107422,8.0,2.033203,1.753906,8.0,1.642578,1.658203,1.713867,1.704102,1.642578,1.450195,1.439453,1.330078,1.566406,0.000000,0.000000,0.0,0.214233,0.567871,2.0,14.585938,9.437500,7.957031,7.910156,18.562500,6.679688,6.113281,7.226562,5.367188,1069,1,1,1,2014,1
32563324,HOBBIES_1_005_CA_1,1441,3,1,0,0,d_1069,1.0,2014-01-01,11349,1,1,0,3.080078,1,1.0,0.0,3.0,1.0,1.0,1.0,3.080078,3.080078,0.000000,9.242188,3.080078,3.080078,2.480469,1.000000,1.414062,4.0,0.750000,4.0,1.000000,1.095703,4.0,1.054688,1.210938,1.286133,1.253906,1.392578,1.274414,1.043945,1.022461,1.216797,1.857422,1.675781,5.0,1.463867,1.426758,5.0,3.080078,3.080078,3.960938,3.861328,9.242188,3.214844,3.148438,3.748047,4.445312,1069,1,1,1,2014,1
32563325,HOBBIES_1_006_CA_1,1442,3,1,0,0,d_1069,0.0,2014-01-01,11349,1,1,0,1.000000,1,1.0,0.0,1.0,1.0,2.0,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.714355,0.755859,2.0,1.036133,10.0,0.966797,1.760742,10.0,0.944824,1.607422,1.000000,0.816406,0.821289,0.856934,1.005859,1.422852,1.972656,1.857422,2.544922,7.0,2.357422,2.857422,9.0,0.714355,0.966797,1.000000,0.816406,2.000000,1.005859,1.422852,1.972656,2.371094,1069,1,1,1,2014,1


In [31]:
data[(data['year'] == 2014) & (data['dayofyear'] < 143) ].to_pickle('train1_1.pkl')
data[(data['year'] == 2014) & (data['dayofyear'] >= 143) & (data['dayofyear'] < 171)].to_pickle('val1_1.pkl')
print("*"*50)
data[(data['year'] == 2015) & (data['dayofyear'] < 143) ].to_pickle('train2_1.pkl')
data[(data['year'] == 2015) & (data['dayofyear'] >= 143) & (data['dayofyear'] < 171)].to_pickle('val2_1.pkl')
print("*"*50)
data[(data['year'] == 2016) & (data['dayofyear'] < 144-56) ].to_pickle('train3_1.pkl')
data[(data['year'] == 2016) & (data['dayofyear'] >= 144-56) & (data['dayofyear'] < 144-28)].to_pickle('val3_1.pkl')
print("*"*50)
data[(data['year'] == 2016) & (data['dayofyear'] >= 144-28) & (data['dayofyear'] < 144)].to_pickle('testpart1_1.pkl')
data[(data['year'] == 2016) & (data['dayofyear'] >= 144) & (data['dayofyear'] < 172)].to_pickle('testpart2_1.pkl')

**************************************************
**************************************************
**************************************************


In [ ]:
#df.to_hdf('merged.hdf','mydata',mode='w')